In [1]:
import numpy as np
import random

# 初始化参数
N = 100  # 序列长度
n_max = 20  # 最大考虑的距离
beta = 3  # 衰减系数
T_initial = 1.0  # 初始温度
alpha = 0.0001  # 温度衰减系数

# 固定的 mu_A 和 mu_B
num_A = 40
num_B = N - num_A
mu_A = num_A / N
mu_B = num_B / N

# 初始化序列
sequence = np.array(['A'] * num_A + ['B'] * num_B)
np.random.shuffle(sequence)

# 计算频率函数
def calculate_frequencies(seq):
    P_AA, P_AB, P_BB = np.zeros(n_max), np.zeros(n_max), np.zeros(n_max)
    for n in range(1, n_max + 1):
        for i in range(N - n):
            pair = (seq[i], seq[i + n])
            if pair == ('A', 'A'):
                P_AA[n - 1] += 1
            elif pair in [('A', 'B'), ('B', 'A')]:
                P_AB[n - 1] += 1
            elif pair == ('B', 'B'):
                P_BB[n - 1] += 1
    P_AA /= (N - n_max)
    P_AB /= (N - n_max)
    P_BB /= (N - n_max)
    return P_AA, P_AB, P_BB

# 计算目标函数 E
def calculate_energy(P_AA, P_AB, P_BB):
    E = 0
    for n in range(n_max):
        E += ((P_AA[n] - mu_A**2)**2 + 
              (P_AB[n] - 2 * mu_A * mu_B)**2 + 
              (P_BB[n] - mu_B**2)**2) * (n + 1)**beta
    return E

# 初始状态
P_AA, P_AB, P_BB = calculate_frequencies(sequence)
E_current = calculate_energy(P_AA, P_AB, P_BB)
T = T_initial  # 初始温度
iteration = 0  # 迭代计数器

print(f"Initialized. Energy: {E_current}, A: {num_A}, B: {num_B}")


Initialized. Energy: 178.29737500000007, A: 40, B: 60


In [2]:
# 多重扰动策略：小扰动和大扰动
def perturb(sequence):
    if random.random() < 0.5:
        # 小扰动：交换两个随机位置
        i, j = random.sample(range(N), 2)
        sequence[i], sequence[j] = sequence[j], sequence[i]
    else:
        # 大扰动：随机选择一个子序列并反转
        start = random.randint(0, N - 5)
        end = start + random.randint(2, 5)
        sequence[start:end] = sequence[start:end][::-1]
    return sequence

# 统计 A 和 B 的数量
def count_atoms(sequence):
    return np.sum(sequence == 'A'), np.sum(sequence == 'B')


In [3]:
# 执行一次 MCMC 迭代
def mcmc_step(sequence, E_current, T, iteration):
    # 执行扰动
    new_sequence = perturb(sequence.copy())

    # 计算新的频率和能量
    P_AA, P_AB, P_BB = calculate_frequencies(new_sequence)
    E_new = calculate_energy(P_AA, P_AB, P_BB)

    # 判断是否接受新解
    if E_new < E_current or np.exp((E_current - E_new) / T) > np.random.rand():
        sequence = new_sequence
        E_current = E_new

    # 更新温度（慢速退火）
    T = T_initial / (1 + alpha * iteration)

    # 打印状态
    if iteration % 1000 == 0:
        count_A, count_B = count_atoms(sequence)
        print(f"Iteration {iteration}, Energy: {E_current}, A: {count_A}, B: {count_B}")

    return sequence, E_current, T


In [6]:
# 继续执行若干次迭代
num_steps = 200000  # 每次执行的迭代次数

for _ in range(num_steps):
    sequence, E_current, T = mcmc_step(sequence, E_current, T, iteration)
    iteration += 1

print(f"Completed {iteration} iterations. Current Energy: {E_current}")
print("Optimized Sequence:", "".join(sequence))
import pandas as pd

# 计算最终的频率
P_AA, P_AB, P_BB = calculate_frequencies(sequence)

# 创建频率表
data = {
    "n": list(range(1, n_max + 1)),
    "P_n^AA": P_AA,
    "P_n^AB": P_AB,
    "P_n^BB": P_BB
}
df = pd.DataFrame(data)

# 打印频率表
print("\nFrequency Table:")
print(df)


Iteration 400000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 401000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 402000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 403000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 404000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 405000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 406000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 407000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 408000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 409000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 410000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 411000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 412000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 413000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 414000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 415000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 416000, Energy: 70.55018749999998, A: 40, B: 60
Iteration 4170

In [10]:
# 保存当前状态
state = {
    'sequence': sequence,
    'E_current': E_current,
    'T': T,
    'iteration': iteration
}
np.save('mcmc_state.npy', state)
print("State saved.")


State saved.


In [ ]:
# 加载保存的状态
state = np.load('mcmc_state.npy', allow_pickle=True).item()
sequence = state['sequence']
E_current = state['E_current']
T = state['T']
iteration = state['iteration']

print(f"State loaded. Resuming from iteration {iteration}, Energy: {E_current}")
